A notebook to run stage 2 analysis of the P1490 dataset

In this script we take the output of the preprocessing pipeline and run a first-level GLM on it using nilearn (per individual participant).

1. Parse the events file to get the onset times and trial types.
2. Parse the output_images.txt file to get the paths to the preprocessed images.
3. Create a FirstLevelModel object and fit it to the data.
4. Optionally compute a simple contrast.
5. Optionally produce a report.
6. Return a dictionary of relevant output paths.

In [ ]:
# Import necessary libraries
import os
from joblib import dump
from joblib import load
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from nilearn.plotting import plot_stat_map
from nilearn.image import load_img, threshold_img
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import FirstLevelModel
from nilearn.input_data import NiftiMasker
from nilearn.plotting import plot_design_matrix
from nilearn.plotting import plot_stat_map
from nilearn.plotting import view_img
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.second_level import SecondLevelModel
from nilearn.plotting import plot_stat_map
from nilearn.masking import compute_epi_mask
from nilearn.maskers import NiftiMasker
from nilearn.plotting import view_img_on_surf
import pandas as pd
from preparefMRI_functions import resample_TR
print('Done importing libraries')

In [ ]:
# directories etc.
pptID = 'R6688' # for this script we run one ppt at a time

base_dir = '/scratch/groups/Projects/P1490'
data_dir = 'Data' # directory of all participants' data
#nifti_dir='nifti' # A subdirectory of the data_dir in each ppt folder
nipype_workdir = 'nipype_workdir' # A subdirectory of the data_dir for each ppt

fslDir = os.environ['FSLDIR']
#fslDir='/raid/toolbox/fsl/'
#fslDir='/groups/labs/wadelab/toolbox/fsl6/'
fslDataDir = os.path.join(fslDir,'data/standard/')
mni_template = os.path.join(fslDataDir,'MNI152_T1_2mm.nii.gz')

t1_dir = 'freesurfer' # directory where recon-all T1 data is
t1_folder = f'{pptID}_fs7/mri'
t1_image = os.path.join(base_dir, t1_dir, t1_folder, 'T1.nii.gz')

parfile_dir = 'parfiles' # A subdirectory of the base_dir
# In here we expect to find the parfiles for the fMRI runs in .par format : tsv with columns time, event, duration
# If duration is not specified, it will be set to 12 seconds
all_parfiles=['color_1.par','color_2.par','color_3.par','color_4.par','color_5.par']
parfile_to_fmriFiles_indices=[1,2,3,4,5] # Which fmri files go with which parfiles. we need this incase we don't want to use all parfiles.
all_event_files = [os.path.join(base_dir,data_dir,pptID,parfile_dir, f) for f in all_parfiles]  

In [ ]:
# define functions
def plot_glm_matrices(glm_list):
    """
    This function plots the design matrices for each Generalized Linear Model (GLM) in the given list.
    
    Parameters:
    glm_list (list): A list of GLM objects.
    """
    # Create a single figure to contain all subplots
    fig = plt.figure(figsize=(12, 8))
    
    # Loop through the GLMs and plot their design matrices as subplots
    for i, thisG in enumerate(glm_list):
        # Extract the design matrix from the current GLM
        dm = thisG.design_matrices_[0]
    
        # Add a subplot for each design matrix in a 2x2 grid of subplots
        ax = fig.add_subplot(2, 2, i + 1)  
        plot_design_matrix(dm, ax=ax)  # Use nilearn's plot_design_matrix function to plot the design matrix
    
    # Adjust subplot layout for better visualization
    plt.tight_layout()
    
    # Show the single figure with all subplots
    plt.show()

def parse_event_tsv(events_tsv):
    print('Events TSV:', events_tsv)
    import pandas as pd
    """
    Read columns: time, event, duration
    Return a DataFrame in the format Nilearn expects:
    columns = [onset, duration, trial_type, ...]
    """
    df = pd.read_csv(events_tsv, sep='\t', header=None)
    # Rename columns to match what Nilearn's FirstLevelModel expects
    # by default: onset, duration, trial_type

    # Set the column names to be 'onset' and 'trial_type'
    # add a new column 'duration' with a default value of 12

    # Set the first column name to be onset
    df.columns = ['onset', 'trial_type']
    df['duration'] = 12

    print(f"Unique trial types in {events_tsv}: {df['trial_type'].unique()}")
    
    return df

The list of processed files for each ppt is held in output_images.txt

These are the locations of the final aligned files from the registration nipype pipeline. Use these for the GLMs.

Use nilearn to compute L1 GLMs for each nifti file.

Loop through the fmri files and the parfiles and run the GLM for each pair once, right after preprocessing.

This has been split below and need to be run seperately or the kernel will crash in YNiC.

In [ ]:
# Read the output_images.txt file to get a list of the preprocessed images
output_images_file = os.path.join(base_dir,data_dir,pptID,nipype_workdir,'output_images.txt')
with open(output_images_file, 'r') as f:
    fmri_files = [line.strip() for line in f.readlines()]
fmri_files.sort()  # Sort the list of files

print('FMRI files:', fmri_files)

In [ ]:
# take the first image to use for viewing later
first_img  = nib.load(fmri_files[0])
# Print the size of this image: x y z t
print('First image shape:', first_img.shape)

# calculate a mask to be used in the GLM later
mask_img   = compute_epi_mask(first_img)      # same geometry as runs

In [ ]:
# run once then can skip
# Here we plot the alignment (structural and functional) for each run
# each run has the same prescription and transformation, so really only need to do one

fmri_data_list=[] 

#for fmri_file, events_tsv in zip(fmri_files, all_event_files):
fmri_file = fmri_files[0]

# Make an image of the mean of the fmri_file for visualisation
print('FMRI file:', fmri_file)

# I have computed a tmean image on the command line as the kernel sometimes crashed on here.
#from pathlib import Path
#orig_path = Path(fmri_file)
#base = orig_path.name.replace(".nii.gz", "")
#tmean_name = base + "_tmean.nii.gz"
#tmean_path = orig_path.with_name(tmean_name)
#fmri_data = load_img(tmean_path)

fmri_data = load_img(fmri_file)
fmri_data_list.append(fmri_data)

# Load in 4D data and average across the 4th dimension (time)
mean_img = fmri_data.get_fdata()
mean_img = mean_img.mean(axis=3)

# Convert the mean image back to a NIfTI image and plot
mean_img = nib.Nifti1Image(mean_img, first_img.affine, first_img.header)
view = plot_stat_map(mean_img, title='Mean image',  display_mode='ortho', cut_coords=(0, 0, 0), threshold=3000)

In [ ]:
# compute a mask of the data
mask_img   = compute_epi_mask(fmri_data)
view = plot_stat_map(mask_img, title='Mean image',  display_mode='ortho', cut_coords=(0, 0, 0), threshold=3000)

In [ ]:
# Compute one GLM per run
results = []
for img_path, events_tsv in zip(fmri_files, all_event_files):
        events = parse_event_tsv(events_tsv)
        glm = FirstLevelModel(
                mask_img       = mask_img,     
                hrf_model      = 'glover',
                noise_model    = 'ar1',
                high_pass  = 0.01,
                smoothing_fwhm = 4,
                t_r            = 0.75,
                minimize_memory=True,
                verbose        = 1)
        glm.fit(img_path, events=events)
        results.append(glm)

# save these results
dump(results, os.path.join(base_dir,data_dir,pptID, 'L1_fits.joblib'))

In [ ]:
# Read in the results from the joblib file
results = load(os.path.join(base_dir,data_dir,pptID,'L1_fits.joblib'))

In [ ]:
# The fitted models are saved in the results list dictionary under the key 'model'
# We can access them like this:
# for result in results:
#     print(result.design_matrices_)

# For each of the models, we can compute a contrast and save it to a file

cNames=['LumDiskIn','LumDiskOut','LMDiskIn','LMDiskOut','SDiskIn','SDiskOut','LumGratingIn','LumGratingOut','LMGratingIn','LMGratingOut','SGratingIn','SGratingOut','ITI']

# Plot the design matrices
for glm in results:
   plot_glm_matrices([glm])

contrasts=[]
for thisG in results:
    # Extract the design matrix from the current GLM
    dm = thisG.design_matrices_[0]
    dm.columns = list(cNames) + list(dm.columns[13:]) 

    # Assign values for the conditions you want to compare (1 v -1)
    v = np.zeros(dm.shape[1])
    cond1_idx  = [dm.columns.get_loc(n) for n in
            ['LumDiskIn','LumDiskOut','LMDiskIn','LMDiskOut','SDiskIn','SDiskOut','LumGratingIn','LumGratingOut','LMGratingIn','LMGratingOut','SGratingIn','SGratingOut',]]
    cond2_idx = [dm.columns.get_loc(n) for n in
            ['ITI']]
    v[cond1_idx]  =  1
    v[cond2_idx] = -1

    # Comput contrast
    con_img = thisG.compute_contrast(v, output_type='z_score')
    contrasts.append(con_img)

# Compute the Level 2 stats - the combined stats on a single subject and session
print('Computing L2')

# Create a second-level model object
second_level_model = SecondLevelModel()
design_matrix = pd.DataFrame({'intercept': [1] * len(results)})

# Fit the model and save
second_level_model = second_level_model.fit(contrasts, design_matrix=design_matrix)
print('Fitted L2')
dump(second_level_model, os.path.join(base_dir,data_dir,pptID,'L2_fits.joblib'))

# Specify the contrast: here, a simple one-sample test
contrast_matrix = np.eye(design_matrix.shape[1])  # Identity matrix of size of the design matrix
z_map = second_level_model.compute_contrast(output_type='z_score')
z_map.to_filename(os.path.join(base_dir,data_dir,pptID,'test_z_map.nii.gz'))     

# We also need to save out the effect size for a group-level analysis
effect_size=second_level_model.compute_contrast(output_type='effect_size')
effect_size.to_filename(os.path.join(base_dir,data_dir,pptID,'test_effMap.nii.gz'))


In [ ]:
# Plot this contrast - this is really just for us, any data presented will be the group data and shown in the next script
zmap_clustThresh = threshold_img(z_map, threshold=2.1, cluster_threshold=5)
view = view_img(zmap_clustThresh, black_bg=True, threshold=2)
view

In [ ]:
# Also display on a surface
view = view_img_on_surf(z_map,darkness=.7,threshold='95%', surf_mesh='fsaverage7',
        vol_to_surf_kwargs=dict(
        radius=3.0, # in mm
        interpolation="linear", # 'linear' or 'nearest'
        kind="ball",  # 'auto', 'line', 'ball', or 'depth'
        n_samples=10  
    ))
view 
